# RedShift Notebook

Notebook Purpose:
- Initiate RedShift cluster and role (with attached policy)
- Validate table creation (create_tables.py) and ETL pipeline (etl.py)
- Delete RedShift cluster and role

In [1]:
import pandas as pd
import boto3
import json

The cell below extracts values from the dwh.cfg file to be used for RedShift cluster and Postgres database configuration

In [9]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_IDENTIFIER")
DWH_DB_NAME                 = config.get("CLUSTER","DWH_DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DWH_DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DWH_DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB_NAME)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB_NAME", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB_NAME, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB_NAME,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhrole


In [10]:
#initiate aws clients

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [11]:
# access udacity's s3 bucket used for data extraction
bucket = s3.Bucket('udacity-dend')

In [12]:
# sample paths to log json files used for extraction
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log-data')]
log_data_files[:10]

['log-data/',
 'log-data/2018/11/2018-11-01-events.json',
 'log-data/2018/11/2018-11-02-events.json',
 'log-data/2018/11/2018-11-03-events.json',
 'log-data/2018/11/2018-11-04-events.json',
 'log-data/2018/11/2018-11-05-events.json',
 'log-data/2018/11/2018-11-06-events.json',
 'log-data/2018/11/2018-11-07-events.json',
 'log-data/2018/11/2018-11-08-events.json',
 'log-data/2018/11/2018-11-09-events.json']

In [13]:
# sample paths to song json files used for extraction
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/A')]
song_data_files[:10]

['song-data/A/A/A/TRAAAAK128F9318786.json',
 'song-data/A/A/A/TRAAAAV128F421A322.json',
 'song-data/A/A/A/TRAAABD128F429CF47.json',
 'song-data/A/A/A/TRAAACN128F9355673.json',
 'song-data/A/A/A/TRAAAEA128F935A30D.json',
 'song-data/A/A/A/TRAAAED128E0783FAB.json',
 'song-data/A/A/A/TRAAAEM128F93347B9.json',
 'song-data/A/A/A/TRAAAEW128F42930C0.json',
 'song-data/A/A/A/TRAAAFD128F92F423A.json',
 'song-data/A/A/A/TRAAAGR128F425B14B.json']

# Create Role

In [14]:
#creates role for redshift cluster
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {
                'Statement': [{'Action': 'sts:AssumeRole',
                               'Effect': 'Allow',
                               'Principal': {'Service': 'redshift.amazonaws.com'}}],
                'Version': '2012-10-17'
            })
    )
except Exception as e:
    print(e)

In [15]:
#attaches s3 read only policy to role
iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']

200

In [16]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

# Create Cluster

In [17]:
try:
    response = redshift.create_cluster(        
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
    """Displays exclusive RedShift cluster information in a pretty pandas dataframe
    
    Arg:
    props -- All RedShift cluster information
    
    """
    
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

# DO NOT RUN CELL BELOW UNLESS ClusterStatus = 'Available'
If do, then recreate cluster

In [22]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

In [ ]:
print('Endpoint: ', DWH_ENDPOINT)
print('ARN: ', DWH_ROLE_ARN)

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName='default',
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

## Validate ETL Pipeline

In [25]:
#sql magic to execute sql queries cells
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB_NAME)
print(conn_string)
%sql $conn_string

# Staging Tables

The below tables are samples of the staging events and songs tables. 

### Staging Events Table

In [ ]:
%sql select * FROM staging_events LIMIT 5;

### Staging Songs Table

In [ ]:
%sql select * FROM staging_songs LIMIT 5;

## Fact and Dimension Tables

The tables below are samples of the fact and dimension tables and their respective table count. 

### Dimension Table: Time

In [ ]:
%sql SELECT COUNT(*) FROM dim_time;

In [ ]:
%sql SELECT * FROM dim_time LIMIT 5;

### Dimension Table: Artists


In [ ]:
%sql SELECT COUNT(*) FROM dim_artists;

In [ ]:
%sql SELECT * FROM dim_artists LIMIT 5;

### Dimension Table: Songs


In [ ]:
%sql SELECT COUNT(*) FROM dim_songs;

In [ ]:
%sql SELECT * FROM dim_songs LIMIT 5;

### Dimension Table: Users


In [ ]:
%sql SELECT COUNT(*) FROM dim_users;

In [ ]:
%sql SELECT * FROM dim_users LIMIT 5;

### Fact Table: Songplay


In [ ]:
%sql SELECT COUNT(*) FROM fact_songplay;

In [ ]:
%sql SELECT * FROM fact_songplay LIMIT 5;

# Delete Cluster to Elimate Unnecessary AWS Charges

In [ ]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)

In [ ]:
try:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
except Exception as e:
    print('Cluster Deleted')
    print('\nError: ', e)

prettyRedshiftProps(myClusterProps)

# Detach and Delete Role

In [ ]:
try:
    iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
except Exception as e:
    print('IAM Role Deleted')
    print('\nError: ', e)